In [1]:
import geopandas as gpd
import numpy as np
from sklearn.neighbors import KernelDensity
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union

In [ ]:
def generate_points_in_gaps(input_shapefile, output_shapefile, num_new_points=1000, bandwidth=0.001):
    # Load the input shapefile
    gdf = gpd.read_file(input_shapefile)
    
    # Extract coordinates of the existing points
    coordinates = np.array([(point.x, point.y) for point in gdf.geometry])
    
    # Create a bounding box around the points
    minx, miny, maxx, maxy = gdf.total_bounds
    bounding_box = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
    
    # Buffer the points slightly to create an area of influence around them
    buffered_points = gdf.buffer(bandwidth).geometry.unary_union
    
    # Identify the gaps (areas not covered by points)
    gaps = bounding_box.difference(buffered_points)
    
    # Fit a Kernel Density model to the coordinates
    kde = KernelDensity(bandwidth=bandwidth)
    kde.fit(coordinates)
    
    # Sample new points
    new_points = []
    while len(new_points) < num_new_points:
        new_coordinates = kde.sample(1)
        new_point = Point(new_coordinates[0])
        
        # Check if the new point is within the gaps
        if gaps.contains(new_point):
            new_points.append(new_point)
    
    # Create a new GeoDataFrame with the generated points
    new_gdf = gpd.GeoDataFrame(geometry=new_points, crs=gdf.crs)
    
    # Save the new points to the output shapefile
    new_gdf.to_file(output_shapefile, driver='ESRI Shapefile')

In [ ]:
input_shapefile = "input_buildings.shp"
output_shapefile = "output_filled_points.shp"
generate_points_in_gaps(input_shapefile, output_shapefile)